In [1]:
import sys
sys.path.append("../venv/lib/python3.7/site-packages/")
from bokeh.io import output_file, show, save
from bokeh.models import GeoJSONDataSource
from bokeh.plotting import figure
from bokeh.sampledata.sample_geojson import geojson
from bokeh.tile_providers import get_provider, Vendors
from bokeh.models import ColumnDataSource, HoverTool
import json
from IPython.display import IFrame
import math
#from ast import literal_eval
#import requests
#from urllib.parse import urljoin

In [2]:
#lift from
#https://towardsdatascience.com/
#exploring-and-visualizing-chicago-transit-data-using-pandas-and-bokeh-part-ii-intro-to-bokeh-5dca6c5ced10
def merc(Coords):

    lat = Coords['lat']
    lng = Coords['lng']
    
    r_major = 6378137.000
    x = r_major * math.radians(lng)
    scale = x/lng
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    return (x, y)

In [3]:
origin = {'lat':41.795,'lng':-87.60} #your location here, Hyde Park, Chicago, US

In [4]:
def map_plot(coords, origin = None, title = None):
    """Plot multiple points in a world map using array of dict coordinates.

    Keyword arguments:
    coords -- array of dict objects containing lat,lng,color 
              eg. [{lat: 41.795, lng: -87.60, color: 'green'}]
    origin -- coordinates for your origin location (default None)
    """
    
    if origin:
        o = merc(origin)
        x_origin = [o[0]]
        y_origin = [o[1]]

    tile_provider = get_provider(Vendors.CARTODBPOSITRON)

    # TODO: enable hover tool
    #hover = HoverTool(tooltips=[
    #    ("station", "@stationname"),
    #    ("ridership","@ridership")
    #])

    #source = ColumnDataSource(data=dict(
    #                        x=list(Merged['coords_x']), 
    #                        y=list(Merged['coords_y']),
    #                        ridership=list(Merged['monthtotal']),
    #                        sizes=list(Merged['circle_sizes']),
    #                        stationname=list(Merged['STATION_NAME'])))

    p = figure(x_range=(-18780000, 18000000), y_range=(-1000000, 7000000),
               x_axis_type="mercator", y_axis_type="mercator", 
               plot_width = 980, plot_height = 500, title = title)
               #tools=[hover, '','wheel_zoom','save']
    
    p.add_tile(tile_provider)

    if origin:
        p.circle(x=x_origin, y=y_origin, size=10, color="black")

    ###
    # plot multiple lines coming from the origin to dest. coordinates
    #
    if origin:
        for coord in coords:
            c = merc(coord)
            c_x = [c[0]]
            c_y = [c[1]]
            #print(c)
            #print(coord['color'])
            p.multi_line(xs=[[x_origin, c_x]], 
                         ys=[[y_origin, c_y]],
                         color=[coord['color']],
                         line_width=2)
    ###
    # only plot point coordinates
    #
    else:
        for coord in coords:
            c_x = [c[0]]
            c_y = [c[1]]
            p.circle(x=c_x, y=c_y, size=10, color=coord['color'])

    output_file("tile1.html")
    save(p)
    return IFrame(src='./tile1.html', width=1024, height=500)
    

coords=[{'lat':41.775,'lng':-88.101,'color':'green'}]
map_plot (coords, origin, title='TCP/UDP End Points')

(-9807358.458378194, 5127334.89537035)
green


In [ ]:
#r = requests.request("GET", "http://192.168.100.1/geoip/8.8.8.8")
#r.json()